In [0]:
import requests
import json
import time
from datetime import datetime
from typing import Dict, Any, List

#  API Setup
api_key = dbutils.secrets.get(scope="zillowsecrets", key="rapidapi_key")
headers = {
    "x-rapidapi-key": api_key,
    "x-rapidapi-host": "zillow-com1.p.rapidapi.com"
}
url = "https://zillow-com1.p.rapidapi.com/priceAndTaxHistory"

#  ZPID List
zpid_list = [
    20530375, 21305363, 20805648, 20385314, 89144774, 20157954, 19954222, 82582423,
    19972775, 20083366, 19942282, 250224427, 20112609, 447566500, 20573654, 402663195,
    136709605, 20529056, 19899526, 136702401, 20751476, 20779023, 20524417, 95637520,
    95547747, 19978675, 20802440, 20604130, 19963489, 447478820, 20455120, 79797561,
    20169079, 20637164, 20741471, 20529664, 20125341, 135446642, 20032611, 20809679,
    20526802
]

#  Storage Containers
price_history_data: List[Dict[str, Any]] = []
tax_history_data: List[Dict[str, Any]] = []

# Helper Functions
def extract_price_history(zpid: int, history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    return [
        {
            "zpid": zpid,
            "date": event.get("date"),
            "event": event.get("event"),
            "price": event.get("price"),
            "priceChangeRate": event.get("priceChangeRate"),
            "pricePerSquareFoot": event.get("pricePerSquareFoot")
        }
        for event in history
    ]

def extract_tax_history(zpid: int, tax_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    return [
        {
            "zpid": zpid,
            "year": tax.get("time"),
            "value": tax.get("value"),
            "taxPaid": tax.get("taxPaid"),
            "valueIncreaseRate": tax.get("valueIncreaseRate"),
            "taxIncreaseRate": tax.get("taxIncreaseRate")
        }
        for tax in tax_data
    ]

#  Main Loop
for zpid in zpid_list:
    try:
        print(f"🔍 Fetching data for ZPID: {zpid}")
        response = requests.get(url, headers=headers, params={"zpid": zpid})
        response.raise_for_status()

        if response.status_code == 200:
            data = response.json()

            # Extract and append price and tax history
            price_history_data.extend(extract_price_history(zpid, data.get("priceHistory", [])))
            tax_history_data.extend(extract_tax_history(zpid, data.get("taxHistory", [])))

        else:
            print(f" Non-200 response for ZPID {zpid}: {response.status_code}")

    except Exception as e:
        print(f" Error retrieving data for ZPID {zpid}: {e}")

    time.sleep(6)  # Respect API rate limits


print("\nPrice History Sample:")
print(json.dumps(price_history_data[:3], indent=2))

print("\n Tax History Sample:")
print(json.dumps(tax_history_data[:3], indent=2))

In [0]:
from datetime import datetime
import json

# Widget inputs
container_name = dbutils.widgets.get("container_name")
storage_account_name = dbutils.widgets.get("storage_account_name")
base_folder_name = dbutils.widgets.get("base_folder_name")
storage_account_key = dbutils.secrets.get(scope = "zillowsecrets",key="storage_account_key")
base_folder_name = dbutils.widgets.get("base_folder_name")
main_folder_price = dbutils.widgets.get("main_folder_price")
main_folder_tax = dbutils.widgets.get("main_folder_tax")

# Configure access
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

# Construct base path
folder_path_tax = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{base_folder_name}/{main_folder_tax}" 
folder_path_price = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{base_folder_name}/{main_folder_price}"


timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
price_file_name = f"{timestamp}.json"
tax_file_name = f"{timestamp}.json"

# Convert to JSON string


# Write JSON to ADLS Gen2
price_file_path = f"{folder_path_price}/{price_file_name}"
tax_file_path = f"{folder_path_tax}/{tax_file_name}"
# Convert to JSON string
price_json_data = json.dumps(price_history_data, indent=2)
tax_json_data = json.dumps(tax_history_data, indent=2)
dbutils.fs.put(price_file_path, price_json_data, overwrite=True)
dbutils.fs.put(tax_file_path, tax_json_data, overwrite=True)


display(f" Price history JSON written to: {price_file_path}")
display(f" Tax history JSON written to: {tax_file_path}")